<a href="https://colab.research.google.com/github/PatrykCiwinski/Topic_modeling-LDA/blob/main/Topic_modeling(LDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LDA

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [11]:
!gdown 1iboAP6jwWe0FnhYlz97Q_62Udc2aTNp4

Downloading...
From: https://drive.google.com/uc?id=1iboAP6jwWe0FnhYlz97Q_62Udc2aTNp4
To: /content/bbc_text_cls.csv
100% 5.09M/5.09M [00:00<00:00, 151MB/s]


In [12]:
df=pd.read_csv('bbc_text_cls.csv')

In [64]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [76]:
#stop-words
stop_words=

In [69]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [71]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [66]:
def clean_text(headline):
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
  cleaned_text=" ".join(tokens)
  return cleaned_text

In [72]:
df['cleaned_text']=df['text'].apply(clean_text)

In [83]:
vectorizer = TfidfVectorizer(max_features=1000)

In [84]:
X=vectorizer.fit_transform(df['cleaned_text'])

In [85]:
no_topics=df['labels'].nunique()

In [86]:
lda=LatentDirichletAllocation(n_components=no_topics)

In [87]:
z = lda.fit_transform(X)

In [88]:
topics = lda.components_

In [90]:
# Print the topics with their terms
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(topics):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:7]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Topic "+str(index+1)+": ",top_terms_list)

Topic 1:  ['said', 'company', 'firm', 'market', 'year', 'bank', 'price']
Topic 2:  ['film', 'award', 'best', 'star', 'show', 'year', 'band']
Topic 3:  ['mobile', 'people', 'phone', 'technology', 'said', 'game', 'computer']
Topic 4:  ['game', 'player', 'club', 'england', 'match', 'team', 'injury']
Topic 5:  ['said', 'labour', 'election', 'party', 'blair', 'government', 'would']


# We can forecast topics as follows:

Topic 1:business,
Topic 2:entertainment,
Topic 3:tech,
Topic 4:sport,
Topic 5:politics